# core

> Fill in a module description here

In [ ]:
#| default_exp data

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from pathlib import Path
from fastai.basics import *

In [ ]:
#| notest
data_path  = Path('data').parent / 'drive' / 'MyDrive' / 'paath.ai' / 'data'

train_data = np.memmap(data_path / 'train.bin', dtype=np.uint16, mode='r')
val_data   = np.memmap(data_path / 'val.bin', dtype=np.uint16, mode='r')

print(train_data.shape, val_data.shape)

(1003854,) (111540,)


In [ ]:
#| hide
!nbdev_export
#import nbdev; nbdev.nbdev_export()